# Simulation

In [1]:
import pandas as pd
import numpy as np
import json
import random
from collections import Counter

pd.options.display.max_rows = 1_000

### Data

In [2]:
proj = pd.read_csv("../data/projections-generated.csv")
proj.head()

,team,name,position,goals,assists,plus_minus,powerplay_points,shots_on_goal,hits,blocks,wins,save_percentage,goals_against_average,saves,shutouts
0,Edm,Connor McDavid,C,30.810,53.447,3.337,23.950,177.233,25.65,22.5,NaN,NaN,NaN,NaN,NaN
1,Col,Nathan MacKinnon,C,28.867,45.260,8.037,21.863,255.060,36.20,20.8,NaN,NaN,NaN,NaN,NaN
2,Edm,Leon Draisaitl,"C,LW",33.977,48.113,-1.223,23.237,163.860,29.80,16.4,NaN,NaN,NaN,NaN,NaN
3,NYR,Artemi Panarin,LW,24.727,46.163,18.190,15.007,160.253,15.20,13.4,NaN,NaN,NaN,NaN,NaN
4,Was,Alex Ovechkin,LW,34.080,21.130,-0.617,15.117,245.010,143.45,22.4,NaN,NaN,NaN,NaN,NaN


In [3]:
df = pd.read_csv("../data/draft-yahoo_league.csv")
df = df.sort_values("pick")
df["position"] = np.where(df["position_yahoo"].isin(["G", "D"]), df["position_yahoo"], "F")
df = df.rename(columns={"pick": 'adp'})
df.head()

,team,age,name,position_yahoo,rollup,vorp,vorn,round,adp,rank,arbitrage,target,position
4,Edm,23.0,Connor McDavid,C,69.3,14.6,7.3,1.0,1.0,5.0,-4.0,True,F
1,Col,25.0,Nathan MacKinnon,C,73.0,18.3,11.0,1.0,2.0,2.0,0.0,True,F
8,Edm,25.0,Leon Draisaitl,"C,LW",65.8,12.6,6.5,1.0,3.0,9.0,-6.0,True,F
11,NYR,29.0,Artemi Panarin,LW,63.3,10.0,4.0,1.0,4.0,12.0,-8.0,True,F
5,Was,35.0,Alex Ovechkin,LW,67.5,14.2,8.2,1.0,5.0,6.0,-1.0,False,F


### Players

In [4]:
players = df[["name", "position", "adp", "vorp"]].to_dict(orient="records")
players[:5]

[{'name': 'Connor McDavid', 'position': 'F', 'adp': 1.0, 'vorp': 14.6},
 {'name': 'Nathan MacKinnon', 'position': 'F', 'adp': 2.0, 'vorp': 18.3},
 {'name': 'Leon Draisaitl', 'position': 'F', 'adp': 3.0, 'vorp': 12.6},
 {'name': 'Artemi Panarin', 'position': 'F', 'adp': 4.0, 'vorp': 10.0},
 {'name': 'Alex Ovechkin', 'position': 'F', 'adp': 5.0, 'vorp': 14.2}]

### Pool Settings

In [5]:
TEAMS = 12
SLOTS = {'F': 6, 'D': 4, 'G': 2}
BENCH = 4
BENCH = 0
PICKS = (sum(SLOTS.values()) + BENCH) * TEAMS 

CATEGORIES = [
    'goals', 
    'assists', 
    'plus_minus', 
    'powerplay_points', 
    'shots_on_goal', 
    'hits', 
    "wins", 
    "save_percentage", 
    "goals_against_average",
    "shutouts"
]

### Draft Order

In [6]:
def snake(low, high, x):
    k = (high - low + 1)
    return k - int(abs(x % (2*k) + low - k - 0.5))

draft_order = [snake(1, 12, x) for x in range(PICKS)]
draft_order[:24]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]

### Team Class

In [7]:
class Team:
    
    SLOTS = Counter({'F': 6, 'D': 4, 'G': 2})
    
    def __init__(self, number, pref_slot=0.8, sort_by="adp", sort_reverse=False):
        self.players = []
        self.number = number
        self.pref_slot = pref_slot
        self.sort_by = sort_by
        self.sort_reverse = sort_reverse
      
    
    def __repr__(self): 
        return f"Team({self.number})"

    
    @property
    def slots_to_fill(self):
        """Slots left to fill"""
        slots_to_fill = self.SLOTS.copy()
        if self.players:
            slots_to_fill -= Counter([player["position"] for player in self.players])
        return slots_to_fill
    
    
    def pick(self, available):
        """Simulate pick"""
        player = self._strategy(available)
        self.players.append(player)
        available.remove(player)
        return player
    
    
    def _strategy(self, available):
        """Draft strategy"""
        ranked = sorted(available, key=lambda x: x[self.sort_by], reverse=self.sort_reverse)
        
        # random pick from the top 5 available
        choice_player = random.choices(
            population=ranked[:5], 
            weights=[0.60, 0.20, 0.10, 0.05, 0.05],
            k=1
        )[0]
        
        # chance to ignore slot preference (1-pref_slot)
        if random.uniform(0, 1) > self.pref_slot:
            return choice_player

        # if there are no more slots to fill, pick top
        if not self.slots_to_fill: 
            return choice_player

        # otherwise loop to get the first required slot player
        for player in ranked:
            if self.slots_to_fill.get(player["position"], 0) > 0: 
                return player

In [8]:
class MyTeam:
    def __init__(self, number):
        self.number = number
        self.players = []
        self.round = 1
        
    def pick(self, available):
        print(f"\n=== Round {self.round} ===\n")
        
        slots_to_fill = Counter(SLOTS.copy())
        if self.players:
            slots_to_fill -= Counter([player["position"] for player in self.players])
            print(f"Slots to fill: {slots_to_fill}\n")
        
        if self.players:
            current = [(p["name"], p["position"]) for p in self.players]
            print(current)
        
        ranked = sorted(available, key=lambda x: x["vorp"], reverse=True)[:100]
        
        print("\nAvailable:")
        for i, player in enumerate(ranked):
            print(i, player["name"], player["position"])
            
        pi = int(input("\nPick: "))
        player = ranked[pi]
        
        print(f"\nSelected: {player}")
        self.players.append(player)
        available.remove(player)
        self.round += 1
        return player

### Simulate

In [9]:
# seed teams
teams = dict()
for number in range(1, TEAMS+1):
    team = Team(number)
    teams[number] = team

# randomize "my" draft pick
me = int(random.uniform(1, TEAMS+1) // 1)
teams[me] = MyTeam(me)
print(f"Team({me})")

# run simulation
available = players.copy()
for i, team in enumerate(draft_order):
    teams[team].pick(available)
    
# "my" picks
teams[me].players

Team(11)

=== Round 1 ===


Available:
0 Tuukka Rask G
1 Robin Lehner G
2 Roman Josi D
3 Victor Hedman D
4 John Carlson D
5 Dougie Hamilton D
6 Carter Hart G
7 Connor Hellebuyck G
8 Anton Khudobin G
9 Mika Zibanejad F
10 Cale Makar D
11 Steven Stamkos F
12 Mikko Rantanen F
13 Brad Marchand F
14 Neal Pionk D
15 Max Pacioretty F
16 Brent Burns D
17 Kris Letang D
18 Alex Pietrangelo D
19 Igor Shesterkin G
20 Philipp Grubauer G
21 Jordan Binnington G
22 Torey Krug D
23 Sidney Crosby F
24 Andrei Svechnikov F
25 David Pastrnak F
26 Evgeni Malkin F
27 J.T. Miller F
28 Mitchell Marner F
29 Shea Theodore D
30 Jake Guentzel F
31 Morgan Rielly D
32 Matthew Tkachuk F
33 Darcy Kuemper G
34 Shea Weber D
35 Sebastian Aho F
36 Blake Wheeler F
37 Mark Stone F
38 Tony DeAngelo D
39 Zach Werenski D
40 Ryan Ellis D
41 Miro Heiskanen D
42 Ryan Pulock D
43 Patrik Laine F
44 Gabriel Landeskog F
45 Mark Giordano D
46 Ilya Samsonov G
47 Jeff Petry D
48 Brady Tkachuk F
49 Patrice Bergeron F
50 Jaroslav Halak G



Pick: 10

Selected: {'name': 'David Pastrnak', 'position': 'F', 'adp': 45.0, 'vorp': 3.7}

=== Round 5 ===

Slots to fill: Counter({'F': 4, 'D': 2, 'G': 2})

[('Mika Zibanejad', 'F'), ('Roman Josi', 'D'), ('Dougie Hamilton', 'D'), ('David Pastrnak', 'F')]

Available:
0 Anton Khudobin G
1 Neal Pionk D
2 Kris Letang D
3 Philipp Grubauer G
4 Torey Krug D
5 Shea Theodore D
6 Darcy Kuemper G
7 Shea Weber D
8 Tony DeAngelo D
9 Zach Werenski D
10 Ryan Ellis D
11 Miro Heiskanen D
12 Ryan Pulock D
13 Mark Giordano D
14 Jeff Petry D
15 Jaroslav Halak G
16 Seth Jones D
17 Adam Fox D
18 Charlie McAvoy D
19 Mikhail Sergachev D
20 Darnell Nurse D
21 Erik Karlsson D
22 Ivan Provorov D
23 Drew Doughty D
24 Colton Parayko D
25 Rasmus Ristolainen D
26 Rasmus Dahlin D
27 Jonathan Marchessault F
28 Jacob Trouba D
29 Quinn Hughes D
30 Jake Muzzin D
31 Tyson Barrie D
32 Thomas Chabot D
33 Elias Lindholm F
34 Mike Hoffman F
35 John Klingberg D
36 Alexander Edler D
37 Brayden Schenn F
38 Ryan Graves D
39 Aar


Pick: 1

Selected: {'name': 'Tony DeAngelo', 'position': 'D', 'adp': 102.0, 'vorp': 1.5}

=== Round 9 ===

Slots to fill: Counter({'F': 4})

[('Mika Zibanejad', 'F'), ('Roman Josi', 'D'), ('Dougie Hamilton', 'D'), ('David Pastrnak', 'F'), ('Anton Khudobin', 'G'), ('Philipp Grubauer', 'G'), ('Shea Theodore', 'D'), ('Tony DeAngelo', 'D')]

Available:
0 Ryan Ellis D
1 Ryan Pulock D
2 Mark Giordano D
3 Jaroslav Halak G
4 Adam Fox D
5 Mikhail Sergachev D
6 Darnell Nurse D
7 Drew Doughty D
8 Colton Parayko D
9 Rasmus Ristolainen D
10 Jacob Trouba D
11 Jake Muzzin D
12 Thomas Chabot D
13 Alexander Edler D
14 Ryan Graves D
15 Aaron Ekblad D
16 Oliver Ekman-Larsson D
17 Keith Yandle D
18 Ryan Suter D
19 Elvis Merzlikins G
20 David Perron F
21 Patric Hornqvist F
22 Chris Kreider F
23 Juuse Saros G
24 Ryan Strome F
25 Blake Coleman F
26 Esa Lindell D
27 Matt Grzelcyk D
28 Joonas Korpisalo G
29 Mattias Ekholm D
30 Oliver Bjorkstrand F
31 Kyle Palmieri F
32 Jamie Benn F
33 Matt Dumba D
34 Erik Gus


Pick: 7

Selected: {'name': 'Blake Coleman', 'position': 'F', 'adp': 154.0, 'vorp': -6.0}


[{'name': 'Mika Zibanejad', 'position': 'F', 'adp': 18.0, 'vorp': 7.3},
 {'name': 'Roman Josi', 'position': 'D', 'adp': 29.0, 'vorp': 13.5},
 {'name': 'Dougie Hamilton', 'position': 'D', 'adp': 46.5, 'vorp': 11.7},
 {'name': 'David Pastrnak', 'position': 'F', 'adp': 45.0, 'vorp': 3.7},
 {'name': 'Anton Khudobin', 'position': 'G', 'adp': 72.0, 'vorp': 7.5},
 {'name': 'Philipp Grubauer', 'position': 'G', 'adp': 73.0, 'vorp': 4.6},
 {'name': 'Shea Theodore', 'position': 'D', 'adp': 91.0, 'vorp': 3.0},
 {'name': 'Tony DeAngelo', 'position': 'D', 'adp': 102.0, 'vorp': 1.5},
 {'name': 'Bryan Rust', 'position': 'F', 'adp': 177.0, 'vorp': -4.4},
 {'name': 'Chris Kreider', 'position': 'F', 'adp': 135.0, 'vorp': -5.7},
 {'name': 'Ryan Strome', 'position': 'F', 'adp': 136.0, 'vorp': -6.0},
 {'name': 'Blake Coleman', 'position': 'F', 'adp': 154.0, 'vorp': -6.0}]

In [11]:
df = pd.DataFrame()
for team in teams:
    picks = [player["name"] for player in teams[team].players]
    dt = pd.DataFrame(proj[proj["name"].isin(picks)][CATEGORIES].mean()).T.round(3)
    dt["team"] = team
    df = df.append(dt)
    
df = df.set_index("team")
df.sort_values("goals", ascending=False)

,goals,assists,plus_minus,powerplay_points,shots_on_goal,hits,wins,save_percentage,goals_against_average,shutouts
team,,,,,,,,,,
3,18.162,27.284,1.491,11.541,151.125,61.150,19.630,0.914,-2.665,2.482
11,17.207,25.158,7.380,10.538,159.197,63.970,19.778,0.918,-2.512,2.055
5,17.124,25.902,4.390,11.474,161.776,55.480,15.904,0.914,-2.536,2.342
1,17.048,30.476,3.436,11.672,148.314,59.405,13.244,0.914,-2.636,1.918
12,16.281,28.313,2.477,11.982,144.645,61.760,14.485,0.916,-2.545,2.194
4,15.472,27.553,5.404,10.379,144.315,60.065,20.840,0.915,-2.593,2.328
10,15.449,31.373,0.860,13.040,144.232,43.730,15.682,0.915,-2.528,2.228
8,15.315,26.583,0.000,9.677,142.200,74.367,21.388,0.914,-2.528,2.718
6,15.175,25.919,2.732,11.454,139.049,75.885,18.525,0.918,-2.547,2.638


In [12]:
df.rank(ascending=False).mean(axis=1)

team
1     7.20
2     7.90
3     6.20
4     6.10
5     6.10
6     6.25
7     7.00
8     6.85
9     7.40
10    6.55
11    4.55
12    5.90
dtype: float64

In [26]:
teams[1].players

[{'name': 'Connor McDavid', 'position': 'F', 'adp': 1.0, 'vorp': 14.6},
 {'name': 'Steven Stamkos', 'position': 'F', 'adp': 23.0, 'vorp': 6.9},
 {'name': 'Jake Guentzel', 'position': 'F', 'adp': 24.0, 'vorp': 2.5},
 {'name': 'Johnny Gaudreau', 'position': 'F', 'adp': 48.0, 'vorp': -2.2},
 {'name': 'Brady Tkachuk', 'position': 'F', 'adp': 49.0, 'vorp': -0.5},
 {'name': 'Ryan Nugent-Hopkins', 'position': 'F', 'adp': 70.0, 'vorp': -6.0},
 {'name': 'Miro Heiskanen', 'position': 'D', 'adp': 71.0, 'vorp': 0.6},
 {'name': 'Ivan Provorov', 'position': 'D', 'adp': 95.0, 'vorp': -1.8},
 {'name': 'John Klingberg', 'position': 'D', 'adp': 98.0, 'vorp': -3.8},
 {'name': 'Ben Bishop', 'position': 'G', 'adp': 119.0, 'vorp': -10.1},
 {'name': 'Ryan Ellis', 'position': 'D', 'adp': 121.0, 'vorp': 1.0},
 {'name': 'Sergei Bobrovsky', 'position': 'G', 'adp': 130.0, 'vorp': -20.1}]